In [54]:
from itertools import product
class Node:
    def __init__(self, prob, symbol, left=None, right=None):
        # probability of symbol
        self.prob = prob

        # symbol 
        self.symbol = symbol

        # left node
        self.left = left

        # right node
        self.right = right

        # tree direction (0/1)
        self.code = ''

# Recibe la señal de audio

In [61]:
# Hay 2 formas de recibir la señal, por input o bien abriendo un archivo de sonido

#Desde un sonido, analizar fft 
import scipy.io.wavfile as waves
import numpy as np
import matplotlib.pyplot as plt
import scipy.fft as fourier
import pickle
import copy

audio_completo='sonidoFinal5min.wav'

Fs,data=waves.read(audio_completo)
Audio_m=data[:,0]

def generacionfrec_ventanas(audio,ts):
    #Inicia filtro (se debe usar sincronización)
    #Quitar los primeros elementos que poseen ceros del arreglo
    aux=audio
    #Filtro de Header
    while aux[0]<2:
        aux=aux[1:]

    tamaño_ventana=int(44100*ts)
    frecuencia=[]
    
    # Detección y almacenamiento del texto
    while len(aux)>tamaño_ventana:
        #Obtener el FFT de la primera ventana tomando los primeros 44100*0.08 elementos
        ventana=aux[0:tamaño_ventana]
        L=len(ventana)
        #Fourier
        gk=fourier.fft(ventana)
        M_gk=abs(gk)
        M_gk=M_gk[0:L//2]
        F=(Fs/L)*np.arange(0,L//2)

        #obtener el maximo de M_gk y buscar su pareja en el eje x

        max=np.max(M_gk)
        frecuencia.append(F[np.where(M_gk==max)])
        aux=aux[tamaño_ventana:]
        #Si se llega al header, implica que se terminó la transmisión del texto
        if len(frecuencia)>13:
            if np.mean(frecuencia[-12:])>12300:
                break
    #Filtramos los valores del header que quedaron en el header
    frec_text=[]
    for x in frecuencia:
        if x<10000:
            frec_text.append(x)

    frec_r=[]
    frec_g=[]
    frec_b=[]
    #Eliminamos audio vacio del final
    while aux[-1]<100:
        aux=aux[:-1]
    # Detección y almacenamiento de la imagen
    while len(aux)>=tamaño_ventana:
        
        #Obtener el FFT de la ventana 
        ventana=aux[0:tamaño_ventana]
        L=len(ventana)
        #Fourier
        gk=fourier.fft(ventana)
        M_gk=abs(gk)
        M_gk=M_gk[0:L//2]
        F=(Fs/L)*np.arange(0,L//2)

        #obtener los 3 máximos de M_gk y buscar su pareja en el eje x

        max=np.sort(M_gk)[-3:]
        lim_r=[300,3335]
        lim_g=[3336,6685]
        lim_b=[6686,9900]
        for i in max:
            a=F[np.where(M_gk==i)]
            if lim_r[0] <= a <= lim_r[1]:
                frec_r.append(a)
            if lim_g[0] <= a <= lim_g[1]:
                frec_g.append(a)
            if lim_b[0] <= a <= lim_b[1]:
                frec_b.append(a)

        frecuencia.append(F[np.where(M_gk==max)])
        aux=aux[tamaño_ventana:]

    return [frec_r,frec_g,frec_b,frec_text]

frecuenciasAudio=generacionfrec_ventanas(Audio_m,0.02)

with open("dic_freq_r.pickle", "rb") as f:
    dic_freq_r = pickle.load(f)
    freq_r = list(dic_freq_r.values())

with open("dic_freq_g.pickle", "rb") as f:
    dic_freq_g = pickle.load(f)
    freq_g = list(dic_freq_r.values())

with open("dic_freq_b.pickle", "rb") as f:
    dic_freq_b = pickle.load(f)
    freq_b = list(dic_freq_r.values())

with open("dic_freq_text.pickle", "rb") as f:
    dic_freq_text = pickle.load(f)
    freq_text = list(dic_freq_r.values())

with open("tree_r.pickle", "rb") as f:
    tree_r = pickle.load(f)

with open("tree_g.pickle", "rb") as f:
    tree_g = pickle.load(f)

with open("tree_b.pickle", "rb") as f:
    tree_b = pickle.load(f)
with open("tree_text.pickle", "rb") as f:
    tree_text = pickle.load(f)


def filtroaproximador(requerida,actual):

    a=requerida
    b=actual
    for i in range(len(b)):
        diferencia=1000
        iteracion=-10
        valor=b[i]
        for j in range(len(a)):
            diferencia_local=abs(a[j]-valor)

            if diferencia_local<diferencia:
                diferencia=diferencia_local
                iteracion=j
        
        b[i]=a[iteracion]
    return b

ftext=filtroaproximador(freq_text,frecuenciasAudio[3])
fr=filtroaproximador(freq_r,frecuenciasAudio[0])
fg=filtroaproximador(freq_g,frecuenciasAudio[1])
fb=filtroaproximador(freq_b,frecuenciasAudio[2])
def binario_generate(dic,list):
    binario=[]
    for x in list:
        for key, value in dic.items():
            if x == value:
                binario.append(key)
    return binario

binariotext=binario_generate(dic_freq_text,ftext)
binarior=binario_generate(dic_freq_r,fr)
binariog=binario_generate(dic_freq_g,fg)
binariob=binario_generate(dic_freq_b,fb)



#fig,ax=plt.subplots()
# plt.figure(figsize=(20,10))
# plt.plot(n,Audio_m)
# plt.xlabel('Tiempo(s)')
# plt.ylabel('Audio')
# plt.show()
#fig,bx=plt.subplots()
# plt.figure(figsize=(20,10))
# plt.plot(F,M_gk)
# plt.xlabel('Frecuencia(Hz)', fontsize='14')
# plt.ylabel('Amplitud FFT', fontsize='14')
# plt.xlim(0,2000)
# plt.show()


<ipython-input-61-d3704a91348c>:86: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  frecuencia.append(F[np.where(M_gk==max)])


In [64]:
def Huffman_Decoding(encoded_data, huffman_tree):
    tree_head = huffman_tree
    decoded_output = []
    for x in encoded_data:
        if x == '1':
            huffman_tree = huffman_tree.right   
        elif x == '0':
            huffman_tree = huffman_tree.left
        try:
            if huffman_tree.left.symbol == None and huffman_tree.right.symbol == None:
                pass
        except AttributeError:
            decoded_output.append(huffman_tree.symbol)
            huffman_tree = tree_head
        
    string = ''.join([str(item) for item in decoded_output])
    return string

 

In [68]:
listpixelstext=Huffman_Decoding(''.join(binariotext),tree_text)
listpixelsr=Huffman_Decoding(''.join(binarior),tree_r)
listpixelsg=Huffman_Decoding(''.join(binariog),tree_g)
listpixelsb=Huffman_Decoding(''.join(binariob),tree_b)

print(listpixelstext)



yrPiN cKPes,si Pe1y  P2,n
